<a href="https://colab.research.google.com/github/jailsonnetodev/PISI3/blob/main/Grupo_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando o dataset US Used cars dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [ ]:
df = pd.read_csv('data/usedcars_usa.csv')
df.shape

In [ ]:
df.describe()

#### Dicionario de dados

In [ ]:
data_dict = {
    'vin': 'Número de identificação do veículo - manter',
    'back_legroom': 'Espaço para as pernas no banco traseiro - usar a média',
    'bed': 'Caçamba - descartar (muitos valores nulos)',
    'bed_height': 'Altura da caçamba - descartar (muitos valores nulos)',
    'bed_length': 'Comprimento da caçamba - descartar (muitos valores nulos)',
    'body_type': 'Tipo de carroceria - usar a moda',
    'cabin': 'Cabine - descartar (muitos valores nulos)',
    'city': 'Cidade - manter',
    'city_fuel_economy': 'Consumo de combustível na cidade - usar a média',
    'combine_fuel_economy': 'Consumo combinado de combustível - descartar',
    'daysonmarket': 'Dias no mercado - manter',
    'dealer_zip': 'CEP do revendedor - manter',
    'description': 'Descrição - manter',
    'engine_cylinders': 'Cilindros do motor - usar a moda',
    'engine_displacement': 'Cilindrada do motor - usar a média',
    'engine_type': 'Tipo de motor - usar a moda',
    'exterior_color': 'Cor exterior - usar a moda',
    'fleet': 'Frota - usar a moda',
    'frame_damaged': 'Chassi danificado - usar a moda',
    'franchise_dealer': 'Concessionária franqueada - manter',
    'franchise_make': 'Marca da franquia - usar a moda e tratar valores None',
    'front_legroom': 'Espaço para as pernas no banco dianteiro - usar a média',
    'fuel_tank_volume': 'Volume do tanque de combustível - usar a média',
    'fuel_type': 'Tipo de combustível - usar a moda',
    'has_accidents': 'Histórico de acidentes - usar a moda e converter para booleano',
    'height': 'Altura - usar a média',
    'highway_fuel_economy': 'Consumo de combustível na estrada - usar a média',
    'horsepower': 'Potência - usar a média',
    'interior_color': 'Cor interior - usar a moda',
    'isCab': 'É caminhonete (cabine) - usar a moda e converter para booleano',
    'is_certified': 'É certificado - manter',
    'is_cpo': 'Certificado de proprietário anterior (CPO) - manter',
    'is_new': 'É novo - manter',
    'is_oemcpo': 'Certificado pelo fabricante (OEM CPO) - manter',
    'latitude': 'Latitude - manter',
    'length': 'Comprimento - usar a média',
    'listed_date': 'Data de listagem - manter',
    'listing_color': 'Cor da listagem - manter',
    'listing_id': 'ID da listagem - manter',
    'longitude': 'Longitude - manter',
    'main_picture_url': 'URL da imagem principal - manter',
    'major_options': 'Opções principais - manter',
    'make_name': 'Nome da marca - manter',
    'maximum_seating': 'Capacidade máxima de assentos - converter para inteiro e usar a média',
    'mileage': 'Quilometragem - usar a média',
    'model_name': 'Nome do modelo - manter',
    'owner_count': 'Número de proprietários - usar a média',
    'power': 'Potência - usar a moda',
    'price': 'Preço - manter',
    'salvage': 'Recuperado - usar a moda e converter para booleano',
    'savings_amount': 'Valor economizado - manter',
    'seller_rating': 'Avaliação do vendedor - usar a média',
    'sp_id': 'ID do vendedor - manter',
    'sp_name': 'Nome do vendedor - manter',
    'theft_title': 'Título de roubo - usar a moda e converter para booleano',
    'torque': 'Torque - usar a moda',
    'transmission': 'Transmissão - usar a moda',
    'transmission_display': 'Exibição da transmissão - usar a moda',
    'trimId': 'ID da versão - manter',
    'trim_name': 'Nome da versão - usar a moda',
    'vehicle_damage_category': 'Categoria de danos do veículo - descartar (todos os valores nulos)',
    'wheel_system': 'Sistema de rodas - usar a moda',
    'wheel_system_display': 'Exibição do sistema de rodas - usar a moda',
    'wheelbase': 'Entre-eixos - usar a moda e converter para float',
    'width': 'Largura - usar a média',
    'year': 'Ano - manter'
}

#### Preprocessamento e transformacao dos dados .parquet

In [ ]:
def gerar_amostra(file_path,amostra, num_amostras=100000):
    if os.path.exists(amostra):
        print(f"O arquivo Parquet '{amostra}' já existe.")
    else:
        print(f"Gerando amostra de {num_amostras} registros e salvando como {amostra}...")
        df = pd.read_csv(file_path, low_memory=False, skiprows=lambda i: i > 0 and i % (3000000 // num_amostras) != 0)
        df.to_csv(amostra,index=False)
        print(f"Amostra gerada e salva como '{amostra}'.")



file_path = 'usedcars_usa.csv'
amostra = 'usedcars_usa100k.csv'
gerar_amostra(file_path, amostra, num_amostras=100000)
df = pd.read_csv('usedcars_usa100k.csv')



drop_columns = ['vin','bed','bed_height','bed_length','cabin','combine_fuel_economy','dealer_zip','description','is_certified','is_cpo','is_oemcpo','latitude','listing_id','longitude','main_picture_url','sp_id','trimId','vehicle_damage_category','major_options']

#Dropar as colunas especificas
def drop_columns_from_df(df, columns_to_drop):
  df = df.drop(columns=columns_to_drop)
  return df
df = drop_columns_from_df(df, drop_columns)


#tratar outlier com o metodo IQR
def drop_outliers(df, columns, k=1.5):
    for column in columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        df[column] = df[column].clip(lower=q1 - k * iqr, upper=q3 + k * iqr)
    return df

# prompt: funcao para extrair o numero da coluna maximum_seating e converter para inteiro , atribuir zero em caso de valores nulos , depois tratar os valores nulos e zero com a media

def extract_number_to_int(df, column):
  df[column] = df[column].str.extract('(\d+)').astype(float).fillna(0).astype(int)
  mean_value = df[column].mean()
  df[column] = df[column].replace(0, mean_value)
  df[column] = df[column].astype(int)
  return df
df = extract_number_to_int(df, 'maximum_seating')


extract_number = ['back_legroom','height','front_legroom','fuel_tank_volume','length','width','wheelbase']

# prompt: funcao para extrair apenas o numero de cada coluna acima da lista e converter pra float

def extract_number_to_float(df, columns, decimal_places=1):
    for column in columns:
        df[column] = (
            df[column]
            .str.extract('(\d+\.\d+|\d+)')
            .astype(float)
            .round(decimal_places)
        )
    drop_outliers(df, columns, k=1.5)
    return df
df = extract_number_to_float(df, extract_number, decimal_places=1)


col_categories_process = ['has_accidents','body_type','engine_cylinders','engine_type','exterior_color','fleet','frame_damaged','franchise_make','fuel_type','has_accidents','interior_color','isCab','theft_title','torque','transmission','transmission_display','trim_name','wheel_system','wheel_system_display','wheelbase','power','salvage']

# prompt: funcao para usar a moda passando as colunas acima

def impute_mode(df, columns):
  for column in columns:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)
  return df

df = impute_mode(df, col_categories_process)



converter_int = ['owner_count','horsepower','city_fuel_economy','engine_displacement','highway_fuel_economy','seller_rating']


def convert_to_int(df, colunas):
    for coluna in colunas:
        # Substituir valores nulos pela média da coluna
        media = df[coluna].mean()
        df[coluna].fillna(media, inplace=True)

        # Converter a coluna para inteiro
        df[coluna] = df[coluna].astype(int)

    return df

df = convert_to_int(df, converter_int)


col_media_process = ['back_legroom','front_legroom','fuel_tank_volume','height','length','mileage','seller_rating','width']


# prompt: funcao para preencher com o valor da media as colunas acima

def impute_mean(df, columns):
  for column in columns:
    mean_value = df[column].mean()
    df[column] = df[column].fillna(mean_value)
  return df

df = impute_mean(df, col_media_process)



#TRADUZINDO TODAS AS COLUNAS EM PT-BR PARA MELHOR ENTENDIMENTO NO PROCESSO DE ANALISE


# Dicionário de tradução das colunas
traducoes = {
    'back_legroom': 'espaco_banco_traseiro',
    'body_type': 'tipo_carroceria',
    'city': 'cidade',
    'city_fuel_economy': 'consumo_cidade',
    'daysonmarket': 'dias_no_mercado',
    'engine_cylinders': 'cilindros_motor',
    'engine_displacement': 'cilindradas_motor',
    'engine_type': 'tipo_motor',
    'exterior_color': 'cor_exterior',
    'fleet': 'frota',
    'frame_damaged': 'chassi_danificado',
    'franchise_dealer': 'concessionaria_franqueada',
    'franchise_make': 'marca_da_franquia',
    'front_legroom': 'espaco_banco_dianteiro',
    'fuel_tank_volume': 'volume_tanque',
    'fuel_type': 'tipo_combustivel',
    'has_accidents': 'historico_acidente',
    'height': 'altura',
    'highway_fuel_economy': 'consumo_estrada',
    'horsepower': 'cavalo_de_potencia',
    'interior_color': 'cor_interior',
    'isCab': 'ee_cabine',
    'is_new': 'ee_novo',
    'length': 'comprimento',
    'listed_date': 'data_listagem',
    'listing_color': 'cor_listagem',
    'make_name': 'nome_marca',
    'maximum_seating': 'maximo_assentos',
    'mileage': 'quilometragem',
    'model_name': 'nome_modelo',
    'owner_count': 'qtd_proprietarios',
    'power': 'potencia',
    'price': 'preco',
    'salvage': 'recuperado',
    'savings_amount': 'valor_economizado',
    'seller_rating': 'avaliacao_vendedor',
    'sp_name': 'nome_vendedor',
    'theft_title': 'titulo_roubo',
    'torque': 'torque',
    'transmission': 'transmissao',
    'transmission_display': 'exibicao_transmissao',
    'trim_name': 'nome_versao',
    'wheel_system': 'sistema_rodas',
    'wheel_system_display': 'exibicao_sistema_rodas',
    'wheelbase': 'entre_eixos',
    'width': 'largura',
    'year': 'ano',
}

# Renomear as colunas usando o dicionário de traduções
df = df.rename(columns=traducoes)

def categorizar_daysonmarket(df):
    bins = [0, 36, 83, 185, 365, float('inf')]
    labels = ['ate-36 dias', '36-83 dias', '83-185 dias', '185-365 dias', '> 365 dias']
    df['dias_no_mercado_label'] = pd.cut(df['dias_no_mercado'], bins=bins, labels=labels, right=False)
    return df

df = categorizar_daysonmarket(df)



def tratar_booleans(df):
    col_bool = df.select_dtypes(include=[bool]).columns
    df[col_bool] = df[col_bool].astype(int)

    return df

df = tratar_booleans(df)


#OBTENDO TODAS AS COLUNAS NUMERICAS


def get_numeric_columns(df):
    # Seleciona colunas que são do tipo int64 ou float64
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    return numeric_columns

col_numerics = get_numeric_columns(df)


#DEFININDO COLUNAS QUE MESMO TENDO VALORES DISCREPANTES SERAO MANTIDOS POIS SE TRATAM DE OUTLIERS NATURAIS E SERA REPROCESSADO NO MODELO COM E SEM
exclude_outliers = ['maximo_assentos','dias_no_mercado','qtd_proprietarios','avaliacao_vendedor','frota','chassi_danificado','concessionaria_franqueada','historico_acidente','ee_cabine','ee_novo','recuperado','titulo_roubo']


#REDEFININDO AS COLUNAS QUE SERAO APLICADAS A FUNÇÃO DE TRTAMENTO DE OUTLIERS
new_col_numerics = [ col for col in col_numerics if col not in exclude_outliers]



#APLICANDO OUTLIERS

def drop_outliers(df, columns, k=1.5,):
    for column in columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        df[column] = df[column].clip(lower=q1 - k * iqr, upper=q3 + k * iqr)
    return df
df = drop_outliers(df,new_col_numerics,k=1.5)






#transformando em .parquet e gerando amostras menores
new_data = df.to_parquet('usedcars_usa.parquet')


def random_parquet(path: str, num: int) ->None:
  data = pd.read_parquet(path)
  new_data = data.sample(num, replace=False)
  num2 = ''.join(reversed(''.join(reversed(f'{num}')).replace('000','k')))
  new_data.to_parquet(path.replace('.',f'{num2}.'))


for i in [20000,50000,100000]:
  random_parquet('usedcars_usa.parquet',i)



df = pd.read_parquet('usedcars_usa100k.parquet')

## Analise Exploratoria

In [ ]:
data = pd.read_parquet('data/usedcars_usa.parquet')


col1,col2 = st.columns([0.3,0.8])

with col1:
    top_categorics = st.slider('Selecione quantidade de marcas que deseja visualizar:', 0, 20, 5)
with col2:
    data_filtered = top_categories(
    data = data,
    top=top_categorics,
    label='nome_marca'
    )
    new_fig = px.bar(data_filtered, x='nome_marca', y='preco')
    new_fig


group_make =  data.groupby(['nome_marca'])['preco'].mean().reset_index().sort_values(by='preco',ascending=False)



fig =plt.figure(figsize = (25 , 10))
sns.heatmap(data.select_dtypes('float' , 'int').corr() , annot = True)
plt.xticks(rotation = 45);
st.pyplot(fig)


df_ano = data.groupby('ano')['dias_no_mercado'].mean().reset_index()
fig = px.line(df_ano, x='ano', y='dias_no_mercado',
              title='Relação entre ano de fabricação e tempo de permanência')
st.plotly_chart(fig)


df_acidentes = data.groupby('historico_acidente')['dias_no_mercado'].mean().reset_index()
fig2 = px.bar(df_acidentes, x='historico_acidente', y='dias_no_mercado',
             title='Impacto de histórico de acidentes no tempo de venda')
st.plotly_chart(fig2)



df_cor = data.groupby('cor_exterior')['dias_no_mercado'].mean().reset_index()

datafiltered = top_categories(
    data=df_cor,
    top=10,
    label='cor_exterior'
)
fig3 = px.bar(datafiltered, x='cor_exterior', y='dias_no_mercado',
             title='Relação entre cor do veículo e tempo de venda')
st.plotly_chart(fig3)



df_combustivel = data.groupby('tipo_combustivel')['preco'].mean().reset_index()
fig4 = px.bar(df_combustivel, x='tipo_combustivel', y='preco', title='Preferência por tipo de combustível')
title=('Relação entre cor do veículo e tempo de venda')
st.plotly_chart(fig4)


df_veiculos_ano = data.groupby(['ano','dias_no_mercado_label']).size().reset_index(name='total_veiculos')
fig5 = px.bar(df_veiculos_ano, x='ano', y='total_veiculos', title='Total de veículos por ano', color='dias_no_mercado_label')
st.plotly_chart(fig5)




df_preco_ano = data.groupby('ano')['preco'].mean().reset_index()
fig6 = px.line(df_preco_ano, x='ano', y='preco', title='Preço médio por ano')
st.plotly_chart(fig6)



seats_df = data.groupby(['maximo_assentos' , 'tipo_carroceria'])['tipo_carroceria'].count().to_frame().rename(columns = {'tipo_carroceria':'Count'}).reset_index()
fig7 = px.bar(seats_df, x="tipo_carroceria", y="Count", animation_frame="maximo_assentos", animation_group="maximo_assentos",
            color="Count")
fig7["layout"].pop("updatemenus") # optional, drop animation buttons
st.plotly_chart(fig7)


fig8 = px.box(data,x='quilometragem',y='dias_no_mercado_label')
fig8



group_by_year = data.groupby(['ano','dias_no_mercado_label']).agg({'preco': np.mean}).reset_index()
fig9 = px.line(group_by_year, x='ano', y='preco', color='dias_no_mercado_label')
fig9


group_km =  data.groupby(['dias_no_mercado'])['quilometragem'].size().reset_index()
fig10 = px.line(group_km, x='dias_no_mercado',y='quilometragem')
fig10


group_make =  data.groupby(['nome_marca'])['preco'].size().reset_index().sort_values(by='preco',ascending=False)

fig11 = px.bar(group_make, x='nome_marca', y='preco')
fig11



group_ano_km_preco = data.groupby(['ano','quilometragem'])['preco'].mean().reset_index()
fig12 = px.bar(group_ano_km_preco, x="ano", y=["quilometragem", "preco"], barmode='group',
            title="Total de Veículos e Preço Médio por Ano")
st.plotly_chart(fig12)

data_make = top_categories(
    data=data,
    top=5,
    label='nome_marca'
)


fig = px.bar(data_make, x='nome_marca', y='dias_no_mercado', color='ano',
             title='Tempo de permanência no mercado por modelo e ano')
fig

## Machine Learning

### Feature engineering

### Classificacao

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit  as st
import os
import pickle
from utils.transform_pkl import main
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.classifier import ConfusionMatrix


def table_report(y_test: np.ndarray, previsao: np.ndarray, method:str =''):
  #st.markdown(f'##### Classification report do metodo:  <span style="color: blue">{method}</span>', unsafe_allow_html=True)
  report = classification_report(y_test, previsao, output_dict=True)
  classification_data = pd.DataFrame(report).transpose()
  print(classification_data)

def table_report(y_test: np.ndarray, previsao: np.ndarray, method:str =''):
  #st.markdown(f'##### Classification report do metodo:  <span style="color: blue">{method}</span>', unsafe_allow_html=True)
  report = classification_report(y_test, previsao, output_dict=True)
  classification_data = pd.DataFrame(report).transpose()
  #st.table(classification_data)
  print(classification_data)
def random_forest(
  x_training: np.ndarray, y_training: np.ndarray, x_test: np.ndarray, y_test: np.ndarray
  )-> None:
  st.markdown('### Resultado do machine learning usando o método Random Forest')

  if not(os.path.isfile('random_forest.pkl')):
    obj_random_forest = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    obj_random_forest.fit(x_training, y_training)
    with open('random_forest.pkl', mode='wb') as f:
      pickle.dump(obj_random_forest, f)
  else:
    with open('random_forest.pkl', 'rb') as f:
      obj_random_forest = pickle.load(f)

  prevision_random_forest = obj_random_forest.predict(x_test)
  importances = pd.Series(
    data=obj_random_forest.feature_importances_,
    index= ['espaco_banco_traseiro', 'tipo_carroceria', 'cidade', 'consumo_cidade',
       'cilindros_motor', 'cilindradas_motor', 'tipo_motor',
       'cor_exterior', 'frota', 'chassi_danificado',
       'concessionaria_franqueada', 'marca_da_franquia',
       'espaco_banco_dianteiro', 'volume_tanque', 'tipo_combustivel',
       'historico_acidente', 'altura', 'consumo_estrada', 'cavalo_de_potencia',
       'cor_interior', 'ee_cabine', 'ee_novo', 'comprimento', 'data_listagem',
       'cor_listagem', 'nome_marca', 'maximo_assentos', 'quilometragem',
       'nome_modelo', 'qtd_proprietarios', 'potencia', 'preco', 'recuperado',
       'valor_economizado', 'avaliacao_vendedor', 'nome_vendedor',
       'titulo_roubo', 'torque', 'transmissao', 'exibicao_transmissao',
       'nome_versao', 'sistema_rodas', 'exibicao_sistema_rodas', 'entre_eixos',
       'largura', 'ano']
  )
  important = importances.to_frame()
  important.reset_index(inplace=True)
  important.columns = ['Importância','Feature', ]
  st.markdown('##### Gráfico de Importância de parametros')
  px.bar(data_frame=important, x='Feature', y='Importância', orientation='h', template='plotly_dark')
  table_report(y_test, prevision_random_forest, 'Random Forest')
  confusion_graph(y_test, prevision_random_forest, 'Random Forest')


def KNN(
  x_training: np.ndarray, y_training: np.ndarray, x_test: np.ndarray, y_test: np.ndarray
  )-> None:
  st.markdown('### Resultado do machine learning usando o método KNN')
  if not(os.path.isfile('KNN_data.pkl')):
    obj_knn = KNeighborsClassifier(n_neighbors=10, weights='distance', p=1)
    obj_knn.fit(x_training, y_training)
    with open('KNN_data.pkl', mode='wb') as f:
      pickle.dump(obj_knn, f)
  else:
    with open('KNN_data.pkl', 'rb') as f:
      obj_knn = pickle.load(f)
  prevision_knn = obj_knn.predict(x_test)
  table_report(y_test, prevision_knn, 'KNN')
  confusion_graph(y_test, prevision_knn, 'KNN')


if not(os.path.isfile('data/usedcars_usa.pkl')):
  print('iniciando...')
  main()

with open('data/usedcars_usa.pkl', 'rb') as f:
    X_training, X_test, y_training, y_test = pickle.load(f)


tree_decision(X_training, y_training, X_test, y_test)
random_forest(X_training, y_training, X_test, y_test)
KNN(X_training, y_training, X_test, y_test)

def confusion_graph(y_test, previsao, method:str = ''):
  #st.markdown(f'##### Matriz de Confução do metodo: <span style="color: blue">{method}</span>', unsafe_allow_html=True)
  labels = sorted(list(set(y_test) | set(previsao)))
  cm = pd.DataFrame(0, index=labels, columns=labels)
  for true_label, predicted_label in zip(y_test, previsao):
      cm.loc[true_label, predicted_label] += 1
  #st.table(cm)
  print(cm)


def tree_decision(
  x_training: np.ndarray, y_training: np.ndarray, x_test: np.ndarray, y_test: np.ndarray
  )-> None:
  #st.markdown('### Resultado do machine learning usando o método Árvore de decisão')
  if not(os.path.isfile('tree_decision.pkl')):
    obj_tree_decision = DecisionTreeClassifier(criterion='entropy')
    obj_tree_decision.fit(X_training, y_training)
    with open('tree_decision.pkl', mode='wb') as f:
      pickle.dump(obj_tree_decision, f)
  else:
    with open('tree_decision.pkl', 'rb') as f:
      obj_tree_decision = pickle.load(f)
  prevision_tree = obj_tree_decision.predict(x_test)
  table_report(y_test, prevision_tree,'Árvore de decisão')
  confusion_graph(y_test, prevision_tree, 'Árvore de decisão')

### Regressao

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

def evaluate_models(models, X_train, y_train, X_test, y_test):
    """
    Avalia múltiplos modelos de regressão e retorna uma tabela com as métricas de desempenho.

    :param models: Dicionário de modelos para avaliar { 'nome_modelo': modelo }
    :param X_train: Conjunto de treino (features)
    :param y_train: Conjunto de treino (target)
    :param X_test: Conjunto de teste (features)
    :param y_test: Conjunto de teste (target)
    :return: DataFrame com as métricas de avaliação (MSE, RMSE, MAE, R²)
    """
    results = []
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        results.append({
            'Modelo': model_name,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2
        })
    results_df = pd.DataFrame(results)

    return results_df
models = {
    'Ridge Regression': Ridge(alpha=1.0),
    'Support Vector Machine (RBF Kernel)': SVR(kernel='rbf'),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'XGBoost': XGBRegressor(n_estimators=100),
    'SVM Linear': SVR(kernel='linear')
}

results_df = evaluate_models(models, X_training_regressor, y_training_regressor, X_test_regressor, y_test_regressor)
print(results_df)

### Clusterizacao

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score


X_clusters = df.iloc[:,[4,28,32]].values

scaler = StandardScaler()
X_clusters = scaler.fit_transform(X_clusters)



wcss = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X_clusters)
    wcss.append(kmeans.inertia_)



graph_wcss = px.line(x = range(1,11), y=wcss)
graph_wcss



kmeans_usedcars = KMeans(n_clusters=4, random_state=0)
labels = kmeans_usedcars.fit_predict(X_clusters)


pca = PCA(n_components=2)
X_clusters_pca = pca.fit_transform(X_clusters)


graph_clusters = px.scatter(x=X_clusters_pca[:,0], y= X_clusters_pca[:,1], color=labels)
graph_clusters.show()

colunas_cluster = ['preco', 'quilometragem', 'cavalo_de_potencia', 'consumo_cidade', 'dias_no_mercado']
df_cluster = df[colunas_cluster]

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajuste eps e min_samples conforme necessário
clusters_dbscan = dbscan.fit_predict(df_scaled)

# Adicionar os clusters ao dataset original
df['cluster_dbscan'] = clusters_dbscan

# Contar o número de clusters gerados (-1 é o ruído)
num_clusters = len(set(clusters_dbscan)) - (1 if -1 in clusters_dbscan else 0)
print(f"Número de clusters encontrados: {num_clusters}")

# Avaliar a qualidade dos clusters (somente se houver mais de um cluster)
if num_clusters > 1:
    silhouette_avg_dbscan = silhouette_score(df_scaled, clusters_dbscan)
    print(f"Índice de Silhueta para DBSCAN: {silhouette_avg_dbscan}")

# Visualização dos clusters
plt.figure(figsize=(8, 6))
plt.scatter(df['preco'], df['quilometragem'], c=df['cluster_dbscan'], cmap='viridis', marker='o', s=50)
plt.xlabel('Preço')
plt.ylabel('Quilometragem')
plt.title('Clusters usando DBSCAN')
plt.show()

#### KMeans

In [ ]:
X_clusters = df.iloc[:,[4,28,32]].values

scaler = StandardScaler()
X_clusters = scaler.fit_transform(X_clusters)



wcss = []

for i in range(1,11):
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X_clusters)
    wcss.append(kmeans.inertia_)



graph_wcss = px.line(x = range(1,11), y=wcss)
graph_wcss



kmeans_usedcars = KMeans(n_clusters=4, random_state=0)
labels = kmeans_usedcars.fit_predict(X_clusters)


pca = PCA(n_components=2)
X_clusters_pca = pca.fit_transform(X_clusters)


graph_clusters = px.scatter(x=X_clusters_pca[:,0], y= X_clusters_pca[:,1], color=labels)
graph_clusters.show()


#### DBScan

In [ ]:

colunas_cluster = ['preco', 'quilometragem', 'cavalo_de_potencia', 'consumo_cidade', 'dias_no_mercado']
df_cluster = df[colunas_cluster]

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

dbscan = DBSCAN(eps=0.5, min_samples=5)  # Ajuste eps e min_samples conforme necessário
clusters_dbscan = dbscan.fit_predict(df_scaled)

# Adicionar os clusters ao dataset original
df['cluster_dbscan'] = clusters_dbscan

# Contar o número de clusters gerados (-1 é o ruído)
num_clusters = len(set(clusters_dbscan)) - (1 if -1 in clusters_dbscan else 0)
print(f"Número de clusters encontrados: {num_clusters}")

# Avaliar a qualidade dos clusters (somente se houver mais de um cluster)
if num_clusters > 1:
    silhouette_avg_dbscan = silhouette_score(df_scaled, clusters_dbscan)
    print(f"Índice de Silhueta para DBSCAN: {silhouette_avg_dbscan}")

# Visualização dos clusters
plt.figure(figsize=(8, 6))
plt.scatter(df['preco'], df['quilometragem'], c=df['cluster_dbscan'], cmap='viridis', marker='o', s=50)
plt.xlabel('Preço')
plt.ylabel('Quilometragem')
plt.title('Clusters usando DBSCAN')
plt.show()